In [65]:
#Shahzeb Umer
#21i-0893


import random

# Define classes (Course, Room, Professor, Section) 
class Course:
    def __init__(self, id, name, is_lab=False):
        self.id = id
        self.name = name
        self.is_lab = is_lab

    def get_id(self):
        return self.id

    def get_name(self):
        return self.name

    def is_lab_course(self):
        return self.is_lab

class Room:
    def __init__(self, id, size, is_large):
        self.id = id
        self.size = size
        self.is_large = is_large

    def get_id(self):
        return self.id

    def get_size(self):
        return self.size

    def is_large_room(self):
        return self.is_large

class Professor:
    def __init__(self, id, name):
        self.id = id
        self.name = name

    def get_id(self):
        return self.id

    def get_name(self):
        return self.name

class Section:
    def __init__(self, id, strength):
        self.id = id
        self.strength = strength

    def get_id(self):
        return self.id

    def get_strength(self):
        return self.strength

# Define chromosome representation for the timetable
class TimetableChromosome:
    def __init__(self, courses, rooms, professors, sections):
        self.courses = courses
        self.rooms = rooms
        self.professors = professors
        self.sections = sections
        self.timetable = []

    def initialize(self):
        # Initialize the timetable randomly
        for course in self.courses:
            for section in self.sections:
                # Randomly assign a professor, room, and time slot to each course section
                professor = random.choice(self.professors)
                room = random.choice(self.rooms)
                day = random.randint(0, 4)  # 0-4 represent Monday to Friday
                if course.is_lab_course():
                    # For lab courses, schedule a single session of 3 hours
                    time_slot = random.randint(5, 9)  # Lab session (2:30 PM - 5:30 PM)
                else:
                    # For theory courses, schedule two sessions of 1 hour 20 minutes each
                    time_slot = random.randint(0, 4)  # Morning session (8:30 AM - 2:30 PM)
                self.timetable.append((course, section, professor, room, day, time_slot))

    def crossover(self, other):
        # Perform single-point crossover operation between two chromosomes
        crossover_point = random.randint(1, len(self.timetable) - 1)  # Select a random crossover point
        offspring1 = self.timetable[:crossover_point] + other.timetable[crossover_point:]
        offspring2 = other.timetable[:crossover_point] + self.timetable[crossover_point:]
        return offspring1, offspring2

    def mutate(self):
        # Perform mutation operation on the chromosome by randomly swapping two classes
        index1, index2 = random.sample(range(len(self.timetable)), 2)  # Randomly select two indices
        self.timetable[index1], self.timetable[index2] = self.timetable[index2], self.timetable[index1] 

    def calculate_fitness(self):
        # Calculate the fitness of the timetable chromosome
        conflicts = 0
        for i, slot1 in enumerate(self.timetable):
            for j, slot2 in enumerate(self.timetable):
                if i != j:
                    # Check for conflicts between time slots
                    if slot1[4] == slot2[4] and slot1[5] == slot2[5] and slot1[3] == slot2[3]:
                        conflicts += 1  # Room conflict
                    if slot1[4] == slot2[4] and abs(slot1[5] - slot2[5]) <= 2 and slot1[2] == slot2[2]:
                        conflicts += 1  # Professor conflict

# Constraint 1: Classes can only be scheduled in free classrooms
                    if slot1[4] == slot2[4] and slot1[5] == slot2[5] and slot1[3] == slot2[3]:
                        conflicts += 1  # Violates constraint 1

                    # Constraint 2: Classroom size constraint
                    if slot1[3].get_size() < slot1[1].get_strength() or slot2[3].get_size() < slot2[1].get_strength():
                        conflicts += 1  # Violates constraint 2

                    # Constraint 3: A professor should not be assigned two different lectures at the same time
                    if slot1[4] == slot2[4] and slot1[5] == slot2[5] and slot1[2] == slot2[2]:
                        conflicts += 1  # Violates constraint 3

                    # Constraint 4: The same section cannot be assigned to two different rooms at the same time
                    if slot1[4] == slot2[4] and slot1[5] == slot2[5] and slot1[1] == slot2[1]:
                        conflicts += 1  # Violates constraint 4

                    # Constraint 5: A room cannot be assigned for two different sections at the same time
                    if slot1[4] == slot2[4] and slot1[5] == slot2[5] and slot1[3] != slot2[3]:
                        conflicts += 1  # Violates constraint 5

                    # # Constraint 6: No professor can teach more than 3 courses
                    # professor_courses = [slot[0] for slot in self.timetable if slot[2] == slot1[2]]
                    # if professor_courses.count(slot1[0]) > 3:
                    #     conflicts += 1  # Violates constraint 6

                    # # Constraint 7: No section can have more than 5 courses in a semester
                    # section_courses = [slot[0] for slot in self.timetable if slot[1] == slot1[1]]
                    # if section_courses.count(slot1[0]) > 5:
                    #     conflicts += 1 # Violates constraint 7

                    # Constraint 8: Each course should have two lectures per week not on the same or adjacent days
                    if slot1[0] == slot2[0] and abs(slot1[4] - slot2[4]) <= 1:
                        conflicts += 1  # Violates constraint 8

                    # Constraint 9: Lab lectures should be conducted in two consecutive slots
                    if slot1[0].is_lab_course() and abs(slot1[5] - slot2[5]) != 1:
                        conflicts += 1  # Violates constraint 9

                    # Constraint 10: All theory classes should be in the morning, and labs in the afternoon
                    if slot1[0].is_lab_course() and slot2[0].is_lab_course():
                        if slot1[5] < 5 or slot2[5] < 5:
                            conflicts += 1  # Violates constraint 10 for lab courses
                    else:
                        if slot1[5] >= 5 or slot2[5] >= 5:
                            conflicts += 1  # Violates constraint 10 for theory courses

                    
                    # Additional Constraints
                    # Constraint: No section can have more than 5 courses in a semester
                    section_courses = [s for s in self.timetable if s[1] == slot1[1]]
                    if len(section_courses) > 5:
                        conflicts += 1  # Violation of the constraint

                    # Constraint: Each course should have two lectures per week not on the same or adjacent days
                    course_lectures = [s for s in self.timetable if s[0] == slot1[0]]
                    if len(course_lectures) != 2:
                        conflicts += 1  # Violation of the constraint
                    else:
                        day_diff = abs(course_lectures[0][4] - course_lectures[1][4])
                        if day_diff == 0 or day_diff == 1:
                            conflicts += 1  # Violation of the constraint

                    # Constraint: Lab lectures should be conducted in two consecutive slots
                    if slot1[0].get_name().lower().startswith('lab') and abs(slot1[5] - slot2[5]) != 1:
                        conflicts += 1  # Violation of the constraint

                    # Constraint: All theory classes should be in the morning, and labs in the afternoon
                    if slot1[0].get_name().lower().startswith('lab') and slot2[0].get_name().lower().startswith('lab'):
                        if slot1[5] < 3 or slot2[5] < 3:
                            conflicts += 1  # Violation of the constraint
                    else:
                        if slot1[5] >= 3 or slot2[5] >= 3:
                            conflicts += 1  # Violation of the constraint

        return -conflicts

# Implement Genetic Algorithm for Timetable Scheduling
class TimetableGeneticAlgorithm:
    def __init__(self, courses, rooms, professors, sections, population_size, mutation_rate, crossover_rate):
        self.courses = courses
        self.rooms = rooms
        self.professors = professors
        self.sections = sections
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.population = []

    def initialize_population(self):
        # Initialize the population with random timetables
        for _ in range(self.population_size):
            chromosome = TimetableChromosome(self.courses, self.rooms, self.professors, self.sections)
            chromosome.initialize()
            self.population.append(chromosome)

    def evolve(self):
        # Evolve the population over generations
        new_population = []
        while len(new_population) < self.population_size:
            # Select parents for crossover operation
            parent1, parent2 = self.select_parents()

            # Apply crossover operation
            if random.random() < self.crossover_rate:
                offspring1, offspring2 = parent1.crossover(parent2)
            else:
                offspring1, offspring2 = parent1.timetable, parent2.timetable

            # Apply mutation operation
            if random.random() < self.mutation_rate:
                offspring1.mutate()
            if random.random() < self.mutation_rate:
                offspring2.mutate()

            # Add offspring to the new population
            new_population.append(TimetableChromosome(offspring1))
            new_population.append(TimetableChromosome(offspring2))

        # Replace the old population with the new one
        self.population = new_population

    def select_parents(self):
        # Select parents for crossover operation
        # Randomly select two parents from the population
        parent1, parent2 = random.sample(self.population, 2)
        return parent1, parent2

    def calculate_fitness(self, chromosome):
        # Calculate the fitness of a timetable chromosome
        return chromosome.calculate_fitness()

    def is_valid(self, chromosome):
        # Check if a timetable chromosome satisfies all constraints
        for i, slot1 in enumerate(chromosome.timetable):
            for j, slot2 in enumerate(chromosome.timetable):
                if i != j:
                    # Constraint 1: Classes can only be scheduled in free classrooms
                    if slot1[4] == slot2[4] and slot1[5] == slot2[5] and slot1[3] == slot2[3]:
                        return False  # Violates constraint 1

                    # Constraint 2: Classroom size constraint
                    if slot1[3].get_size() < slot1[1].get_strength() or slot2[3].get_size() < slot2[1].get_strength():
                        return False  # Violates constraint 2

                    # Constraint 3: A professor should not be assigned two different lectures at the same time
                    if slot1[4] == slot2[4] and slot1[5] == slot2[5] and slot1[2] == slot2[2]:
                        return False  # Violates constraint 3

                    # Constraint 4: The same section cannot be assigned to two different rooms at the same time
                    if slot1[4] == slot2[4] and slot1[5] == slot2[5] and slot1[1] == slot2[1]:
                        return False  # Violates constraint 4

                    # Constraint 5: A room cannot be assigned for two different sections at the same time
                    if slot1[4] == slot2[4] and slot1[5] == slot2[5] and slot1[3] != slot2[3]:
                        return False  # Violates constraint 5

                    # Constraint 6: No professor can teach more than 3 courses
                    professor_courses = [slot[0] for slot in chromosome.timetable if slot[2] == slot1[2]]
                    if professor_courses.count(slot1[0]) > 3:
                        return False  # Violates constraint 6

                    # Constraint 7: No section can have more than 5 courses in a semester
                    section_courses = [slot[0] for slot in chromosome.timetable if slot[1] == slot1[1]]
                    if section_courses.count(slot1[0]) > 5:
                        return False  # Violates constraint 7

                    # Constraint 8: Each course should have two lectures per week not on the same or adjacent days
                    if slot1[0] == slot2[0] and abs(slot1[4] - slot2[4]) <= 1:
                        return False  # Violates constraint 8

                    # Constraint 9: Lab lectures should be conducted in two consecutive slots
                    if slot1[0].is_lab_course() and abs(slot1[5] - slot2[5]) != 1:
                        return False  # Violates constraint 9

                    # Constraint 10: All theory classes should be in the morning, and labs in the afternoon
                    if slot1[0].is_lab_course() and slot2[0].is_lab_course():
                        if slot1[5] < 5 or slot2[5] < 5:
                            return False  # Violates constraint 10 for lab courses
                    else:
                        if slot1[5] >= 5 or slot2[5] >= 5:
                            return False  # Violates constraint 10 for theory courses
        return True


    def print_timetable(self):
        # Find the chromosome with the best fitness
        best_chromosome = max(self.population, key=lambda x: x.calculate_fitness())
        print("Best fittness: ", best_chromosome.calculate_fitness())

        # Sort the timetable by day and then by time slot number
        sorted_timetable = sorted(best_chromosome.timetable, key=lambda x: (x[4], x[5]))

        # Print the timetable for the chromosome with the best fitness
        print("Best Timetable:")
        print("------------------------------------------------------------------------")
        print("Course\t\tSection\t\tProfessor\tRoom\tDay\tTime Slot")
        print("------------------------------------------------------------------------")
        for slot in sorted_timetable:
            course_name = slot[0].get_name()
            section_id = slot[1].get_id()
            professor_name = slot[2].get_name()
            room_id = slot[3].get_id()
            day = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"][slot[4]]
            time_slot = slot[5]
            # Adjust spacing to ensure consistent alignment
            print(f"{course_name.ljust(15)}\t{section_id}\t\t{professor_name.ljust(12)}\t{room_id}\t{day.ljust(15)}\t{time_slot}")
            # print("------------------------------------------------------------------------")
        print("------------------------------------------------------------------------")


# Main program
if __name__ == "__main__":
    # Define example data for courses, rooms, professors, and sections
    courses = [
        Course(1, "Math"),
        Course(2, "Physics"),
        Course(3, "Chemistry"),
        Course(4, "Biology"),
        Course(5, "Computer"),
        Course(6, "Lab Physics", is_lab=True),
        Course(7, "Lab Chemistry", is_lab=True),
        Course(8, "Lab Biology", is_lab=True),
    ]

    rooms = [
        Room(1, 60, False),  # Regular classroom
        Room(2, 120, True),  # Large hall
        Room(3, 60, False),  # Regular classroom
        Room(4, 60, False),  # Regular classroom
    ]

    professors = [
        Professor(1, "Dr. Smith"),
        Professor(2, "Prof. Johnson"),
        Professor(3, "Dr. Garcia"),
        Professor(4, "Prof. Lee"),
    ]

    sections = [
        Section(1, 30),
        Section(2, 40),
        Section(3, 35),
        Section(4, 25),
    ]

    # Initialize the genetic algorithm
    timetable_ga = TimetableGeneticAlgorithm(
        courses, rooms, professors, sections, population_size=100, mutation_rate=0.1, crossover_rate=0.1
    )
    # Initialize population
    timetable_ga.initialize_population()
    timetable_ga.print_timetable()


Best fittness:  -3744
Best Timetable:
------------------------------------------------------------------------
Course		Section		Professor	Room	Day	Time Slot
------------------------------------------------------------------------
Chemistry      	1		Prof. Lee   	4	Monday         	0
Computer       	4		Dr. Smith   	4	Monday         	0
Biology        	1		Prof. Johnson	1	Monday         	1
Physics        	1		Dr. Garcia  	3	Monday         	2
Lab Biology    	2		Prof. Johnson	3	Monday         	5
Lab Chemistry  	4		Prof. Lee   	2	Monday         	9
Math           	1		Prof. Johnson	3	Tuesday        	0
Computer       	3		Prof. Lee   	4	Tuesday        	0
Physics        	2		Dr. Garcia  	4	Tuesday        	2
Chemistry      	2		Prof. Johnson	2	Tuesday        	2
Lab Physics    	2		Dr. Garcia  	3	Tuesday        	6
Math           	2		Prof. Johnson	4	Wednesday      	0
Math           	4		Prof. Lee   	3	Wednesday      	1
Computer       	2		Dr. Smith   	3	Wednesday      	1
Physics        	4		Prof. Johnson	2	We